#### Génération des URL de récupération des séquences et récupération par scrapping

In [79]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
def transformer_sequence(sequence,long):
    nucleotides = np.zeros((4,long))
    cnt = 0
    for lettre in sequence:
        if lettre=='a':
            nucleotides[0,cnt] = 1
            cnt = cnt+1
        elif lettre=='c':
            nucleotides[1,cnt] = 1
            cnt = cnt+1
        elif lettre=='g':
            nucleotides[2,cnt] = 1
            cnt = cnt+1
        else:
            nucleotides[3,cnt] = 1
            cnt = cnt+1
    return nucleotides

In [3]:
def recuperer_sequence(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(4)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    browser.quit()
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("span",{'class':"ff_line"})
    rep = seq[0].text+seq[1].text+seq[2].text+seq[3].text
    rep = ''.join(rep.split())
    return rep

In [4]:
def recuperer_sequence2(url):
    browser.get(url)
    time.sleep(1.3)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("div",{'class':"seq gbff"})
    rep = seq[0].text   
    return rep

In [5]:
def transformer_database(bddrow):
    sequence=recuperer_sequence(bddrow[6])
    sequence=transformer_sequence(sequence,len(sequence))
    sequencebis=recuperer_sequence(bddrow[7])
    sequencebis=transformer_sequence(sequencebis,len(sequencebis))    
    bddrow = np.array([bddrow[18],bddrow[19],bddrow[20],bddrow[21],sequence,bddrow[0],bddrow[1],bddrow[2],bddrow[8],bddrow[10],bddrow[11],bddrow[14],bddrow[16],sequencebis,bddrow[3],bddrow[4],bddrow[5],bddrow[9],bddrow[12],bddrow[13],bddrow[15],bddrow[17]], dtype=object)
    return bddrow

In [6]:
def transformer_database2(bddrow):
    sequence=recuperer_sequence2(bddrow[6])
    sequencebis=recuperer_sequence2(bddrow[7])
    bddrow = np.array([sequence,sequencebis,bddrow[8],bddrow[9]], dtype=object)
    return bddrow

In [28]:
import RNA

def construire_seq_et_structure_bdd(bdd):
    for i,el in enumerate(bdd):
        bdd[i,2] = np.array(RNA.pfl_fold_up(bdd[i,0],1,201,201))[1:,1]
        bdd[i,3] = np.array(RNA.pfl_fold_up(bdd[i,1],1,201,201))[1:,1]
        bdd[i,0] = transformer_sequence(bdd[i,0],201)
        bdd[i,1] = transformer_sequence(bdd[i,1],201)
    return bdd

def nettoyer(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=bdd[i,0].split('\n')[1]+bdd[i,0].split('\n')[2]+bdd[i,0].split('\n')[3]
        bdd[i,1]=bdd[i,1].split('\n')[1]+bdd[i,1].split('\n')[2]+bdd[i,1].split('\n')[3]
    return bdd

def traduire_séquences(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=traduire(bdd[i,0],bdd[i,2])
        bdd[i,1]=traduire(bdd[i,1],bdd[i,3])
    return bdd

def traduire(seq,strand):
    nucleotides = ''
    if strand:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'u'
            elif lettre=='C':
                nucleotides=nucleotides+'g'
            elif lettre=='G':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    else:
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'a'
            elif lettre=='C':
                nucleotides=nucleotides+'c'
            elif lettre=='G':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    return nucleotides

In [31]:
bdd = pd.read_csv("rise_human_transcriptome (copie).csv", sep = "\t",header=None)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
bdd

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,chr12,8949583,8949624,chrX,71300009,71300031,RISE0195929,2.000000e+00,-,+,...,ENSG00000147140,NONO,protein_coding,protein_coding,5UTR,CDS,PARIS,human,HEK293T,27180905
1,chr12,8943493,8943527,chr15,72200479,72200507,RISE0195930,2.000000e+00,-,-,...,ENSG00000067225,PKM,protein_coding,protein_coding,CDS,CDS,PARIS,human,HEK293T,27180905
2,chr12,2795005,2795025,chr2,219219447,219219467,RISE0195931,2.000000e+00,+,-,...,ENSG00000198925,ATG9A,protein_coding,protein_coding,5UTR,3UTR,PARIS,human,HEK293T,27180905
3,chr12,2797743,2797777,chr10,32911609,32911628,RISE0195932,2.000000e+00,+,-,...,ENSG00000150093,ITGB1,protein_coding,protein_coding,CDS,CDS,PARIS,human,HEK293T,27180905
4,chr12,2799176,2799208,chr8,68306526,68306558,RISE0195933,2.000000e+00,+,-,...,ENSG00000248801,C8orf34-AS1,protein_coding,ncRNA,CDS,Exon,PARIS,human,HEK293T,27180905
5,chr12,2803390,2803412,chr1,31425381,31425409,RISE0195934,2.000000e+00,+,+,...,ENSG00000168528,SERINC2,protein_coding,protein_coding,3UTR,CDS,PARIS,human,HEK293T,27180905
6,chr2,72131833,72131852,chr16,30922092,30922115,RISE0195935,2.000000e+00,-,-,...,ENSG00000260852,FBXL19-AS1,protein_coding,ncRNA,3UTR,Exon,PARIS,human,HEK293T,27180905
7,chr4,11429576,11429597,chr5,141241519,141241539,RISE0195936,2.000000e+00,-,-,...,ENSG00000280029,CH17-140K24,protein_coding,ncRNA,5UTR,Exon,PARIS,human,HEK293T,27180905
8,chr4,11428894,11428912,chr1,114536496,114536536,RISE0195937,2.000000e+00,-,+,...,ENSG00000236480,PKMP1,protein_coding,pseudogene,5UTR,Exon,PARIS,human,HEK293T,27180905
9,chr4,11399551,11399569,chr5,115262608,115262633,RISE0195938,2.000000e+00,-,+,...,ENSG00000271797,CTC-428G20,protein_coding,ncRNA,CDS,Exon,PARIS,human,HEK293T,27180905


In [8]:
# bdd convertie panda -> numpy array et seuls les 70 000 premiers RRIs retenus
arrbdd = np.array(bdd)[:70000]

#enlever 'chr' et formater les numéros de chromosomes
for i,el in enumerate(arrbdd[:,0]):
    arrbdd[i,0] = arrbdd[i,0][3:]

for i,el in enumerate(arrbdd[:,0]):
    if arrbdd[i,0] == 'X':
        arrbdd[i,0] = '23'
    if arrbdd[i,0] == 'Y':
        arrbdd[i,0] = '24'
    if arrbdd[i,0] == '1':
        arrbdd[i,0] = '01'
    if arrbdd[i,0] == '2':
        arrbdd[i,0] = '02'
    if arrbdd[i,0] == '3':
        arrbdd[i,0] = '03'
    if arrbdd[i,0] == '4':
        arrbdd[i,0] = '04'
    if arrbdd[i,0] == '5':
        arrbdd[i,0] = '05'
    if arrbdd[i,0] == '6':
        arrbdd[i,0] = '06'
    if arrbdd[i,0] == '7':
        arrbdd[i,0] = '07'
    if arrbdd[i,0] == '8':
        arrbdd[i,0] = '08'
    if arrbdd[i,0] == '9':
        arrbdd[i,0] = '09'

for i,el in enumerate(arrbdd[:,3]):
    arrbdd[i,3] = arrbdd[i,3][3:]

for i,el in enumerate(arrbdd[:,3]):
    if arrbdd[i,3] == 'X':
        arrbdd[i,3] = '23'
    elif arrbdd[i,3] == 'Y':
        arrbdd[i,3] = '24'  
    elif arrbdd[i,3] == '1':
        arrbdd[i,3] = '01'
    elif arrbdd[i,3] == '2':
        arrbdd[i,3] = '02'
    elif arrbdd[i,3] == '3':
        arrbdd[i,3] = '03'
    elif arrbdd[i,3] == '4':
        arrbdd[i,3] = '04'
    elif arrbdd[i,3] == '5':
        arrbdd[i,3] = '05'
    elif arrbdd[i,3] == '6':
        arrbdd[i,3] = '06'
    elif arrbdd[i,3] == '7':
        arrbdd[i,3] = '07'
    elif arrbdd[i,3] == '8':
        arrbdd[i,3] = '08'
    elif arrbdd[i,3] == '9':
        arrbdd[i,3] = '09'


#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'M') & (arrbdd[:,3] != 'M'))
varrbdd = arrbdd[arg_sansM[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = 'true'
    else:
        varrbdd[i,8] = 'false'

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = 'true'
    else:
        varrbdd[i,9] = 'false'


#int str
for i,el in enumerate(varrbdd[:,1]):
    if type(el) == int:
        varrbdd[i,1] = str(el)
for i,el in enumerate(varrbdd[:,2]):
    if type(el) == int:
        varrbdd[i,2] = str(el)
for i,el in enumerate(varrbdd[:,4]):
    if type(el) == int:
        varrbdd[i,4] = str(el)
for i,el in enumerate(varrbdd[:,5]):
    if type(el) == int:
        varrbdd[i,5] = str(el)

#génération position début et fin de séquence (nucléotides)
for i,el in enumerate(varrbdd[:,1]):
    milieu = int((int(varrbdd[i,1])+int(varrbdd[i,2]))/2)
    varrbdd[i,1] = str(milieu-35)
    varrbdd[i,2] = str(milieu+35)
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = str(milieu-35)
    varrbdd[i,5] = str(milieu+35)

# Génération URLs

#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,0]+"?report=genbank&from="+varrbdd[i,1]+"&to="+varrbdd[i,2]+"&strand="+varrbdd[i,8]+".html"
for i,el in enumerate(varrbdd[:,6]):
    varrbdd[i,6] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,0]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,1]+'&to='+varrbdd[i,2]

#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,3]+"?report=genbank&from="+varrbdd[i,4]+"&to="+varrbdd[i,5]+"&strand="+varrbdd[i,9]+".html"
for i,el in enumerate(varrbdd[:,7]):
    varrbdd[i,7] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,3]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,4]+'&to='+varrbdd[i,5]


In [9]:
# Ne garder que les interactions miRNA - mRNA

args = np.argwhere(varrbdd[:,14]=='protein_coding')[:,0]
args2 = np.argwhere(varrbdd[:,15]=='protein_coding')[:,0]
intersect = [i for i in args if i in args2]
intersect = np.array(intersect)
varrbdd = varrbdd[intersect]

### Récupération rapide des séquences à partir de chr_nb start end

In [109]:
# package BioPython
# download genome at ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_24/GRCh38.primary_assembly.genome.fa.gz
from Bio import SeqIO

parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")

dict_fasta = dict([(seq.id, seq) for seq in parser])

ide, begin, end = ['chr12',8949582,8949623]

dict_fasta['chr1'][begin:end].format("fasta").split('\n')[1]

### A partir de là c'est l'ancienne méthode (à conserver au cas où)

On en veut 5000

In [24]:
varrbdd = varrbdd[:5000]

#### On  sauvegarde la base de données dans un fichier csv

In [28]:
browser = webdriver.Chrome()
import csv

writer = csv.writer(open("genomes/database.csv", 'w',newline=''),delimiter="\t")
for i,el in enumerate(varrbdd):
    row = transformer_database2(varrbdd[i])    
    writer.writerow(row)

On cherche les trous dans les données

In [7]:
bdd_5000_trouee = pd.read_csv("genomes/database.csv", sep = "\t",header = None)

In [8]:
bdd_5000_trouee = np.array(bdd_5000_trouee)
l = []
for i,el in enumerate(bdd_5000_trouee):
    if type(bdd_5000_trouee[i,0])==float or type(bdd_5000_trouee[i,1])==float:
        a=0
    elif len(bdd_5000_trouee[i,0])==0 or len(bdd_5000_trouee[i,0])==0:
        a=0
    else:
        l.append(i)
l = np.array(l)
len(l)

4983

In [23]:
bdd_miRNA_mRNA = bdd_5000_trouee[l]

In [26]:
bdd_miRNA_mRNA = nettoyer(bdd_miRNA_mRNA)

In [29]:
bdd_miRNA_mRNA = traduire_séquences(bdd_miRNA_mRNA)

In [47]:
#ver = 'aaacuuucggggucccacagaagccaaugcucaagcauccccaagcuccucuaggcucuagccccugcuccagccacguaccaacauucauggcaaaguucacccggagguccacguccucagcccaggccuccuggacuggguccuugcacagcacagggaagaugccacgguacaggugggccugacgagcugucuggg'
ver = ver[90:111]
len(ver),ver

(21, 'aguucgggcagcgcaauucac')

In [20]:
bdd_apprentissage = construire_seq_et_structure_bdd(bdd_miRNA_mRNA[:15])

### Réseau de neurones basé sur iDeep

In [ ]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
activation='relu',
                 input_shape=s.shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#### A partir de mainenant, on essaye d'obtenir les probabilités d'accesibilité de chaque nucléotide sur une séquence quelconque ARN par exemple 'ACGUUGUCACACGAU'

In [ ]:
import RNA

In [ ]:
help(RNA.pfl_fold)

In [ ]:
len('augcucgucgaucguaguagcucaguacgucaguacgaucgacuacgacuguacguacgacugacguaugcuagcugacucgaucguaucgaucgacugcau')

In [ ]:
a = RNA.pfl_fold_up('augcucgucgaucguaguagcucaguacgucaguacgaucgacuacgacuguacguacgacugacguaugcuagcugacucgaucguaucgaucgacugcau',1,102,102)

In [ ]:
b = np.array(a)
b = b[1:,1]
b